In [231]:
from ultralytics import YOLO
import os
import cv2

In [232]:
video = "C:\\Users\\Fabio\\Desktop\\TestVideo.mp4"
weights = "runs/detect/train11/weights/best.pt"
outputFolder = "C:\\Users\\Fabio\\Desktop\\TestOutput"

In [ ]:
model = YOLO(weights)
cap = cv2.VideoCapture(video)
prevX = 1
lastWasTheSame = False
wiggleRoom = 0.003
fSkip = 7
ret, frame = cap.read()
hImg,wImg,_ = frame.shape



In [234]:
frame_skip = fSkip

frame_count = 0
x = None
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_skip == 0:
        results = model.predict(frame)
        frame_skip = fSkip
        for r in results:
            for box in r.boxes:
                x, y, w, h = box.xywh[0].tolist() # Get the x, y, w, h coordinates.
                xNorm, yNorm, wNorm, hNorm =  x/wImg, y/hImg, w/wImg, h/hImg
                if abs(xNorm - prevX) < wiggleRoom:
                    if not lastWasTheSame:
                        lastWasTheSame = True
                    else:
                        lastWasTheSame = False
                        filename = os.path.join(outputFolder, f"frame_{frame_count}.jpg")
                        cv2.imwrite(filename, frame)
                        frame_skip = 100
                if prevX > xNorm:
                    prevX =  xNorm
    frame_count += 1

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Printheads, 7.7ms
Speed: 2.1ms preprocess, 7.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
